In [17]:
import string 
import re 
import pickle # not used
import pandas as pd 
import time
import numpy as np
import emoji




In [18]:
import pandas as pd
import nltk
import re

In [8]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt') 
nltk.download('wordnet') 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aish21/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/aish21/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/aish21/nltk_data...


In [27]:
df = pd.read_csv("Code/data/reddit/Zoom/zoom_reddit.csv")

display(df)
titles = list(df['Title'])
# print(titles)

FileNotFoundError: [Errno 2] No such file or directory: 'Code/data/reddit/Zoom/zoom_reddit.csv'

In [19]:
class TweetCleaner:
  def __init__(self):
    self.stop_words = set(stopwords.words('english'))
    self.emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':(': 'sad', 'XD': 'laughing',
          ':-(': 'sad', ':-<': 'sad', ':P': 'stuck-out-tongue', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          ':/': 'confused', ':|': 'neutral-face', ":'-)": 'sadsmile', "<3": 'love',
          ":'-)": 'tears-of-happiness'}
    
  def remove_emojis(self, tweet):
    # replace emojis with empty string
    clean_text = emoji.get_emoji_regexp().sub(u'', tweet)
    return clean_text  

  def lowercase(self, tweet):
    ''' Each text is converted to lowercase. '''
    return tweet.lower()
  
  def replace_url(self, tweet):
    ''' Links starting with “Http” or “https” or “www” are replaced by “URL” '''
    url_regex = re.compile(r'(http[s]?://|www\.)\S+')
    return url_regex.sub('URL', tweet)

  def replace_emojis(self, tweet):
    '''Replace emojis by using a pre-defined dictionary containing emojis 
      along with their meaning. (e.g.: “:)” to “EMOJIsmile”) '''
    for emoji in self.emojis.keys():
      tweet = tweet.replace(emoji, "EMOJI" + self.emojis[emoji]) 
    return tweet

  def replace_username(self, tweet):
    ''' Replace @Usernames with the word “USER”. (e.g.: “@Kaggle” to “USER”)'''
    user_regex = re.compile(r'@[^\s]+')
    return user_regex.sub('USER', tweet)  

  def remove_nonalpha(self, tweet):
    ''' Replacing characters except Digits and Alphabets with space.'''
    nonalpha_regex = re.compile(r'[^a-zA-Z0-9]')
    return nonalpha_regex.sub(" ", tweet)
  
  def remove_consecutives(self, tweet):
    ''' 3 or more consecutive letters are 
        replaced by two letters. (e.g.: “Heyyyy” to “Heyy”) '''
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    return re.sub(sequencePattern, seqReplacePattern, tweet)

  def remove_stop_short_words(self, tweet):
    ''' English words that do not add much meaning to a sentence are removed
        and Words with a length of less than two are eliminated.'''
    words = nltk.word_tokenize(tweet)
    words = [word for word in words if word not in self.stop_words and len(word) >= 2]
    return ' '.join(words)

  def lemmatize(self, tweet):
    ''' Converting word to its base form. '''
    tweetwords = ''
    for word in tweet.split():
      word = WordNetLemmatizer().lemmatize(word)
      tweetwords += (word+' ')
    return tweetwords

  def clean_onetweet(self, tweet):
    ''' cleans one single tweet '''
    cleaned = self.lowercase(tweet)
    cleaned = self.replace_url(cleaned)
    cleaned = self.replace_emojis(cleaned)
    cleaned = self.remove_emojis(cleaned)
    cleaned = self.replace_username(cleaned)
    cleaned = self.remove_nonalpha(cleaned)
    cleaned = self.remove_consecutives(cleaned)
    cleaned = self.remove_stop_short_words(cleaned)
    cleaned = self.lemmatize(cleaned)
    return cleaned

  def clean_alltweets(self, df):
    ''' cleans all tweets in the dataframe'''
    df['tweets_processed'] = df['tweet'].apply(self.clean_onetweet)
    df = df.drop(columns=['tweet'])
    df = df.rename(columns={'tweets_processed': 'tweet'})
    return df

In [20]:
clean_titles = []
tweetCleaner = TweetCleaner()
for title in titles:
    ctitle = tweetCleaner.clean_onetweet(title)
    clean_titles.append(ctitle)
    


In [21]:
print(clean_titles)

['barren street bustling stock market virus stricken china ', 'amc either insane amount value circling around drain securityanalysis think ', 'guy think amc stock ', 'amc theater closing 12 week good stock daddy ', 'testing stock amc jeep cylinder head ported unported edelbrock jeep head dyno ', 'bought amc stock guy think ', 'amc stock ', 'top ten selling product great depression converted modern stock ', 'major corporation declared bankruptcy yet seem like close declaring due impact covid 19 ', 'amc entertainment stock jumped today ', 'stock watch today apr 17 2020 top stock short ', 'daily watchlist friday 04 17 20 ', 'stock watch list april 17 2020 ', 'missed amc stock EMOJIsad ', 'daily watchlist monday 04 20 20 ', 'happens buy put option company go bankrupt ', 'personally love taking family theater recently got stock 20 amc plan hold plan open july anyone else recently bought amc thought opinion ', 'daily watchlist tuesday 04 28 20 ', 'amc ban universal ', 'daily watchlist wednes

In [22]:
df['clean_title'] = clean_titles


In [25]:
display(df)

,Unnamed: 0,Title,Timestamp,clean_title
0,0,Barren streets but bustling stock markets in v...,2020-02-11 07:21:38,barren street bustling stock market virus stri...
1,1,$AMC either has an insane amount of value or i...,2020-03-08 21:17:48,amc either insane amount value circling around...
2,2,What do you guys think about AMC stock?,2020-03-16 09:08:03,guy think amc stock
3,3,AMC theaters closing for up to 12 weeks is thi...,2020-03-17 12:43:45,amc theater closing 12 week good stock daddy
4,4,Testing Stock AMC/Jeep 4.0 Cylinder Heads (Por...,2020-03-19 13:14:08,testing stock amc jeep cylinder head ported un...
...,...,...,...,...
1542,1542,What happens to people will less than 10 AMC S...,2022-12-26 11:36:29,happens people le 10 amc stock split
1543,1543,"📰 ""AMC Stock: Is CEO Adam Aron a Hero or a Vil...",2022-12-28 01:48:35,amc stock ceo adam aron hero villain amc share...
1544,1544,(12/28) Wednesday's Stock Market Movers & News,2022-12-28 08:39:59,12 28 wednesday stock market mover news
1545,1545,(12/28) Wednesday's Stock Market Movers & News,2022-12-28 08:41:42,12 28 wednesday stock market mover news


In [26]:
df.to_csv('amc_clean_reddit_data.csv', index=False)
